In [0]:
!pip install imgviz
!pip install labelme
%cd /content/drive/My Drive/Segmentation_lib/JsonByLabelme

     |████████████████████████████████| 7.7MB 3.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for imgviz: filename=imgviz-1.1.0-cp36-none-any.whl size=7674269 sha256=fe54f3ab4e8e6ea49685e07c4c21978307fdf9f309d47ed3b5eceee6cbd5460c
  Stored in directory: /root/.cache/pip/wheels/3e/47/fb/93fb92792d5774b08504fd4e6d872d675018d4327e6b7c32b1
Successfully built imgviz
     |████████████████████████████████| 1.5MB 3.4MB/s 
     |████████████████████████████████| 63.6MB 47kB/s 
     |████████████████████████████████| 256kB 63.3MB/s 
  Created wheel for labelme: filename=labelme-4.2.10-cp36-none-any.whl size=1463176 sha256=5e606c12e26489b046e956e7191143ed39d4a7b1220360d9a460f4db75aecd98
  Stored in directory: /root/.cache/pip/wheels/d4/cb/e9/05593f531ac7ce9f99e5ca036fb7416b3b689b11ddddf8cc96
Successfully built labelme
/content/drive/My Drive/Segmentation_lib/JsonByLabelme


In [0]:
!python Json_to_Png.py --labels labels.txt json_0523 train_dataset

Output directory already exists: train_dataset


In [0]:
%cd /content/drive/My Drive/Segmentation_lib

/content/drive/My Drive/Segmentation_lib


In [0]:
# %tensorflow_version 1.14

!pip install tensorflow-gpu==1.13.1
import tensorflow as tf
print(tf.__version__)

import os
import imageio
from keras.engine import InputSpec  
import matplotlib.pyplot as plt
import numpy as np
import skimage.transform as trans
import sys
sys.path.append('/content/drive/My Drive/Segmentation_lib')

import Unet_model
import data_generator
import train
import output_generator
import callback
from keras import optimizers
from keras import backend as K
%matplotlib inline

%cd
%cd /content/drive/My Drive/Segmentation_lib


img_shape = (512, 512, 3)
img_path = '/content/drive/My Drive/Segmentation_lib/JsonByLabelme/train_dataset/JPEGImages'
seg_path = '/content/drive/My Drive/Segmentation_lib/JsonByLabelme/train_dataset/SegmentationClassPNG'

# a = np.array(seg_files)
# print(a.shape)

from sklearn.model_selection import train_test_split
imgs = os.listdir(img_path)
segs = os.listdir(seg_path)

imgs.sort()
segs.sort()

learn_train, learn_test, ans_train, ans_test = train_test_split(
    imgs, segs, test_size = 0.3, random_state=0
)

input_path = '/content/drive/My Drive/Segmentation_lib/JsonByLabelme/train_dataset/JPEGImages'

imgs_pixcount = 0
segs_pixcount = 0

# for i in range(len(img_files)):

#     img = imageio.imread(f'{img_path}/{img_files[i]}', pilmode='RGB')
#     seg = imageio.imread(f'{seg_path}/{seg_files[i]}', pilmode='RGB')

#     img_pix = np.count_nonzero(img)
#     seg_pix = np.count_nonzero(seg)

#     imgs_pixcount += img_pix
#     segs_pixcount += seg_pix

# Non_seg = imgs_pixcount - segs_pixcount
# seg_prop = segs_pixcount / Non_seg
# Class_weight = 1/seg_prop

# print(Class_weight)

weights_comb = [[1.,1.],[1.,0.1],[1.,0.01],[1.,0.001]]
epochs_comb = [12,24, 36]
optimizer_comb = [optimizers.Adam(lr=5e-4)]
pre_num = [0,5]


steps_per_epoch = 64
validation_steps = 32
batch_size = 4

result_path = '/content/drive/My Drive/Segmentation_lib/result'

for weights in weights_comb:
    for epochs in epochs_comb:
        for optimizer in optimizer_comb:
            print("weights:", weights)
            print("epochs:", epochs)
            print("optimizer:", optimizer)
            train_gen = data_generator.get_datagen(img_path, seg_path, learn_train, ans_train, img_size=img_shape, batch_size=batch_size, train=True)
            test_gen  = data_generator.get_datagen(img_path, seg_path, learn_test, ans_test, img_size=img_shape, batch_size=batch_size, train=False)

            if optimizer==optimizer_comb[0]:
                opt = 'adam'

            else:
                opt = 'rms'

            weight_name = str(weights)+str(epochs)+str(optimizer)+'.hdf5'

            train.run(img_path, seg_path, learn_test, ans_test, imgs, segs, img_shape, weights, train_gen, steps_per_epoch, epochs, test_gen, validation_steps, weight_name, optimizer, batch_size)

            pre_name = str(weights)+str(epochs)+str(opt)+'predict_seg'

            output_generator.run(input_path, weight_name, pre_num, result_path, pre_name)


     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 40.3MB/s 
     |████████████████████████████████| 368kB 59.4MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1


Using TensorFlow backend.


1.13.1
/root
/content/drive/My Drive/Segmentation_lib
Instructions for updating:
Colocations handled automatically by placer.
weights: [1.0, 1.0]
epochs: 12
optimizer: <keras.optimizers.Adam object at 0x7f7d4ae760f0>
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/12
64/64 [==============================] - 45s 700ms/step - loss: 0.8793 - accuracy: 0.9799 - val_loss: 0.6377 - val_accuracy: 0.9920
Epoch 2/12
64/64 [==============================] - 34s 532ms/step - loss: 0.5051 - accuracy: 0.9921 - val_loss: 0.3987 - val_accuracy: 0.9933
Epoch 3/12
64/64 [==============================] - 34s 531ms/step - loss: 0.3794 - accuracy: 0.9919 - val_loss: 0.3287 - val_accuracy: 0.9925
Epoch 4/12
64/64 [==============================] - 34s 532ms/step - loss: 0.3092 - accuracy: 0.9922 - val_loss: 0.2882 - val_accuracy: 0.9908
Epoch 5/12
64/64 [==============================] - 34s 531ms/step - loss: 0.2631 - a

KeyboardInterrupt: ignored

In [0]:
%cd /content/drive/My Drive/Segmentation_lib
import output_generator

input_path = '/content/drive/My Drive/Segmentation_lib/learn'

weight_name = 'weight_final.hdf5'

pre_num = [0, 5]

result_path = '/content/drive/My Drive/Segmentation_lib/result'

pre_name = 'predict_seg'

output_generator.run(input_path, weight_name, pre_num, result_path, pre_name)

/content/drive/My Drive/Segmentation_lib
(512, 512, 3)
0
(512, 512, 3)
1
(512, 512, 3)
2
(512, 512, 3)
3
(512, 512, 3)
4
